## Install environments

In [ ]:
!sudo apt-get update -y > /dev/null
!sudo apt-get install python3.11 python3.11-dev python3.11-distutils libpython3.11-dev > /dev/null
!sudo apt-get install python3.11-venv binfmt-support > /dev/null
!sudo apt-get install python3-pip > /dev/null
!python3 -m pip install --upgrade pip > /dev/null
!python3 -m pip install ipykernel


In [ ]:
# 1. Install Rust Toolchain
!curl --proto '=https' --tlsv1.2 -sSf https://sh.rustup.rs | sh -s -- -y
import os
os.environ['PATH'] += ":/root/.cargo/bin"

# 2. Verify Installation
!rustc --version
!cargo --version

# 3. Install Maturin (Build Backend for qdp-python)
!pip install maturin

In [ ]:
!curl -LsSf https://astral.sh/uv/install.sh | sh

In [ ]:
!nvcc --version


## Install Mahout

In [ ]:
# 1. Clone the repository
!git clone -b dev-qdp https://github.com/apache/mahout.git

# 2. Install Python Dependencies
# We use the requirements file provided in the benchmark folder
%cd /content/mahout/qdp/qdp-python
!uv venv -p python3.11
# Activate the virtual environment and run uv sync and maturin develop
!source .venv/bin/activate && uv sync --group dev
# !source .venv/bin/activate && uv run --python 3.11 maturin develop

In [ ]:
%cd /content/mahout/qdp/qdp-python

# 1. Remove all old wheels to prevent future conflicts
!rm -rf /content/mahout/qdp/target/wheels/*

# 2. Re-run build (so we have exactly one fresh wheel)
!uv run maturin build --interpreter .venv/bin/python

# 3. Install the single new wheel found
!uv pip install /content/mahout/qdp/target/wheels/*.whl --python .venv/bin/python --force-reinstall

!./.venv/bin/python -c "import mahout_qdp; print(mahout_qdp.__file__)"

In [ ]:
!uv pip install -r /content/mahout/qdp/benchmark/requirements.txt --python .venv/bin/python

In [ ]:
!uv pip install matplotlib-inline --python .venv/bin/python

## Run Benchmarks

In [ ]:
!./.venv/bin/python /content/mahout/qdp/benchmark/benchmark_e2e.py --frameworks mahout-parquet mahout-arrow pennylane --qubits 18 --samples 500